# Image Segmentation

> A tool to generate the area of interest maps of images using a variety of image segmentation techniques.

Code was written by Nicholas M. Synovic, Oscar Yanek, and Rohan Sethi

## Setup

### Upgrade Python `pip` Tool

Upgrade the Python `pip` tool to the latest version

In [ ]:
%pip install --upgrade pip

### Install Python libaries via `pip`

Installed libraries are:

- opencv-contrib-python
- progress

In [ ]:
%pip install opencv-contrib-python progress

### Import Dependencies 

In [ ]:
from os import listdir
from os.path import join
from pathlib import PurePath

import cv2
import numpy
from numpy import ndarray
from progress.bar import Bar

### Allow Data to be Loaded From Google Drive

If you wish to load data from Google Drive, uncomment the following lines.

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

## Application

### Read Directory

Function to read a directory and return a list of filepaths from that directory.

In [ ]:
def readDirectory(dir: str) -> list:
    files: list = listdir(dir)
    filepaths: list = [join(dir, f) for f in files]
    return filepaths

### Canny Segmentation

Takes a file path to an image (`imagePath`) and an output folder path (default is `./data`; `outputFolder`) as input.

It then uses the approach outlined in [1](#citations) to estimate the background threshold of the image.

Area of interest maps are saved in `.jpg` format in the `outputFolder` with the following scheme:

- `outputFolder`/FILENAME_cannySegmentation`.jpg`

Where FILENAME is the original name of the file without the extension.

In [ ]:
def cannySegmentation(imagePath: str, outputFolder: str = "data")   ->  None:
    imageName: str = PurePath(imagePath).with_suffix("").name + "_cannySegmentation.jpg"
    outputPath: str = join(outputFolder, imageName)

    image: ndarray = cv2.imread(imagePath)
    imageShape: tuple = image.shape

    grayImage: ndarray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) 
    
    _, threshold = cv2.threshold(grayImage, numpy.mean(grayImage), 255, cv2.THRESH_BINARY_INV)
    edges: ndarray = cv2.dilate(cv2.Canny(threshold, 0, 255), None)

    contours: ndarray = sorted(cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2], key=cv2.contourArea)[-1]

    mask: ndarray = numpy.zeros(imageShape, numpy.uint8)
    
    segmented: ndarray = cv2.drawContours(mask, [contours],-1, 255, -1)
    segmented = cv2.cvtColor(segmented, cv2.COLOR_BGR2GRAY)
    segmented[segmented > 0] = 255    # Convert from grayscale to black and white

    cv2.imwrite(outputPath, segmented)

### Main Method

In [ ]:
def main() -> None:
    dir: str = input("Image directory to analyze: ")
    imagePaths: list = readDirectory(dir)
    
    with Bar(
        "Creating saliency maps of PascalVOC images...", max=len(imagePaths)
    ) as bar:
        imagePath: str
        for imagePath in imagePaths:
            cannySegmentation(imagePath)
            bar.next()


if __name__ == "__main__":
    main()


## Citations

1. John Canny. A computational approach to edge detection. Pattern Analysis and Machine Intelligence, IEEE Transactions on, (6):679–698, 1986.